In [0]:
import dlt
# from pyspark.sql.functions import (
#     col, to_date, count, sum, avg, round,
#     current_timestamp, when, datediff
# )
# from pyspark.sql.types import (
#     StructType, StructField, StringType, IntegerType,
#     DoubleType, TimestampType
# )

import sys
sys.path.append('../.')
from helpers import project_functions

In [0]:
TARGET = "nyctaxi_test.default"
RAW_PATH = "/Volumes/nyctaxi_test/default/simple_volume/nyctaxi_data.csv"

In [0]:
# ------------------------------------------------------------------------------
# DLT BRONZE TABLE
# ------------------------------------------------------------------------------
@dlt.table(
    comment="Bronze table containing raw taxi records.",
    table_properties={"quality": "bronze"}
)
def nyc_taxi_bronze():
    df = (
        spark.readStream
        .format("cloudFiles")
        .option("cloudFiles.format", "csv")
        .option("header", "true")
        .schema(project_functions.get_nyc_schema())
        .load(RAW_PATH)
        .limit(20000)
    )
    return project_functions.clean_bronze(df)

In [0]:
# ------------------------------------------------------------------------------
# DLT SILVER TABLE
# ------------------------------------------------------------------------------
@dlt.table(
    comment="Cleaned and validated Silver table for analytics.",
    table_properties={"quality": "silver"}
)
def nyc_taxi_silver():
    bronze_df = dlt.read_stream("nyc_taxi_bronze")
    validated = project_functions.validate_and_cast_silver(bronze_df)
    enriched = project_functions.enrich_silver(validated)
    return enriched